# Nonlinear dimensionality reduction


## Table of contents
* [0. Libraries and helper functions](#libraries)
* [1. MDS - multidimensional scaling](#mds)
* [2. ISOMAP - swissroll](#isomap)
* [3. LLE - local linear embedding](#lle)
* [4. MVU - maximum variance unfolding](#mvu)

Note that in this notebook the data is given as a matrix of column vectors, i.e.
$$ X = [x_1, \ldots, x_n] \in \mathbb{R}^{d\times n}$$

<a class="anchor" id="libraries"></a>
## 0. Libraries and helper functions

In [ ]:
import math
import numpy as np
import plotly.express as px
import plotly.graph_objects as po

In [ ]:
def what_is(expr_str):
    print(expr_str + ' ==')
    print(eval(expr_str))


<a class="anchor" id="mds"></a>
## 1. MDS - Multidimensional Scaling

In [ ]:
# goal: create a map from a distance table scanned 
# if I recall correctly, I typed/ocr'ed them from the last page of an old road map
cities = ['Aachen', 'Basel', 'Berlin', 'Bremen', 'Dortmund', 
          'Dresden', 'Duesseldorf', 'Emden', 'Erfurt', 
          'Flensburg', 'Frankfurt M.', 'Frankfurt O.', 
          'Garmen-Patenkirchen', 'Goerlitz', 'Hamburg', 
          'Hannover', 'Kassel', 'Koblenz', 'Koeln', 'Leipzig', 
          'Mannheim', 'Muenchen', 'Nuernberg', 'Passau', 
          'Rostock', 'Saarbruecken', 'Salzburg', 'Stuttgart', 
          'Trier', 'Wiesbaden']
scanned_distances = \
'''545
650  875
370  775 400
155  555 495 235
645  745 200 490 515
 90  550 560 285  70 580
375  845 520 140 305 620 290
440  585 300 340 310 215 375 470
625  980 450 275 490 660 540 400 520
255  335 550 445 225 460 225 520 260  650
700  940 105 460 560 180 625 590 370  550 610
700  370 675 835 700 550 680 930 510 1020 480 470
750  840 220 575 610 105 680 700 320  690 560 170 650
480  825 300 130 350 500 400 255 370  160 500 385 860 530
355  680 290 130 215 385 280 260 220  310 350 350 720 470 155
310  525 385 285 165 350 235 390 150  470 200 450 560 450 310 170
165  405 600 410 190 510 145 425 310  670 120 660 550 610 520 390 250
 75  505 580 315 100 570  40 330 370  570 200 640 650 670 430 300 250 105
570  710 190 390 440 115 505 520 140  570 385 255 520 215 440 290 280 440 500
290  270 615 515 300 530 280 600 330  725  85 680 410 630 570 430 270 150 250 460
650  390 590 750 605 460 610 840 420  930 400 650  90 560 780 630 470 490 580 430 350 
470  450 440 585 440 315 445 675 270  795 230 510 260 420 610 470 310 340 410 280 240 170
690  580 630 800 660 465 655 890 460  980 440 700 280 570 820 680 520 550 620 470 440 190 220
650 1000 230 300 515 440 570 425 490  285 740 325 870 470 180 330 560 690 600 380 810 780 630 820
310  265 725 550 330 640 280 560 440  810 200 790 500 740 660 530 380 180 250 570 140 430 370 570 920
800  530 735 910 755 605 770 980 580 1080 540 800 180 540 920 780 625 660 720 570 510 150 320 120 940 600
420  270 630 630 420 510 410 710 350  810 210 700 300 610 660 520 360 280 370 470 135 230 210 400 820 220 390
160  325 715 480 260 635 200 500 430  735 190 780 580 730 590 460 340 140 180 560 180 500 420 620 760 100 700 300
230  350 570 430 210 490 200 480 280  680  40 640 500 590 520 380 220  80 170 410 100 430 260 470 760 160 570 220 150'''


In [ ]:
# convert them into a symmetric numpy matrix
n = len(cities)
D = np.zeros((n, n))
rows = scanned_distances.split('\n')
for i in range(n-1):
    D[i+1,:i+1] = rows[i].split()
D = D + D.T   # symmetrize
print(D[:5,:5])

In [ ]:
# weird indexing that also reverses the ordering in one step (avoiding `flip`)
# this is useful to get the last two columns of the matrix of eigenvectors
# in the correct ordering
a = np.arange(5)
print(a)
a[:-3:-1]

In [ ]:
# run multi-dimensional scaling
from ml_solutions import mds
x = mds(D, 2)
fig = px.scatter(x=x[0], y=x[1], text=cities, title='Germany')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
# change the plot a bit
fig = px.scatter(x=-x[1], y=-x[0], text=cities, title='Germany')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
fig.update_layout(width=800, height=800)
fig.show()

<a class="anchor" id="isomap"></a>
## 2.1 ISOMAP - swissroll

In [ ]:
# some advanced slicing with numpy 
# (we use this in our loop-free implementation of knn_graph)
a = np.arange(15).reshape(3,5)
b = np.array([[1,4],[0,2],[0,4]])
c = np.arange(3).reshape(3,1)
print(a)
print(b)
a[c,b]= 100
print(a)

In [ ]:
# neighborhood graphs
from ml_solutions import distances, knn_graph, eps_graph
# test code
x = np.arange(10).reshape(1,10)
D = distances(x)
what_is('distances(x)')
what_is('knn_graph(D, 2)')
what_is('knn_graph(D, 4)')
what_is('eps_graph(D, 0.5)')
what_is('eps_graph(D, 1.1)')
what_is('eps_graph(D, 2.1)')

In [ ]:
# all pairs-shortest paths
from ml_solutions import floyd_warshall
# test code
x = np.arange(10).reshape(1,10)
D = distances(x)**2
W = knn_graph(D, 3)
what_is('floyd_warshall(W, 2)')
x = np.random.randn(3,4)
W = knn_graph(distances(x), 2)
what_is('floyd_warshall(W, 3)')
what_is('floyd_warshall(W, 2)')

In [ ]:
# generate the swiss roll dataset
def swissroll(n=1000, evenly_sampled=True):
    z = np.random.rand(2, n)
    if evenly_sampled:
        z[0] = np.sqrt(z[0])
    z = 3 * math.pi * z
    r = z[0] + 1.0
    x = np.vstack([r*np.cos(z[0]), 
                   r*np.sin(z[0]), 
                   z[1]])
    return x, z
x, z = swissroll()
px.scatter_3d(x=x[0], y=x[1], z=x[2], color=z[0], 
              color_continuous_scale=px.colors.sequential.Rainbow)

In [ ]:
# see also https://plotly.com/python/network-graphs/
def plot_graph(x, W, color=None):
    n = W.shape[0]
    edges_x, edges_y = [], []
    for i in range(n):
        for j in range(i):
            if 0 < W[i,j] < np.infty:
                edges_x.extend([x[0,i], x[0,j], None])
                edges_y.extend([x[1,i], x[1,j], None])
    edges_trace = po.Scatter(x=edges_x, y=edges_y, mode='lines')
    nodes_trace = po.Scatter(x=x[0],    y=x[1],    mode='markers', 
                             marker=dict(color=color, size=5.0))
    return po.Figure(data=[edges_trace, nodes_trace])
def plot_graph_3d(x, W, color=None):
    n = W.shape[0]
    edges_x, edges_y, edges_z = [], [], []
    for i in range(n):
        for j in range(i):
            if 0 < W[i,j] < np.infty:
                edges_x.extend([x[0,i], x[0,j], None])
                edges_y.extend([x[1,i], x[1,j], None])
                edges_z.extend([x[2,i], x[2,j], None])
    edges_trace = po.Scatter3d(x=edges_x, y=edges_y, z=edges_z, mode='lines')
    nodes_trace = po.Scatter3d(x=x[0],    y=x[1],    z=x[2],    mode='markers', 
                               marker=dict(color=color, size=5.0))
    return po.Figure(data=[edges_trace, nodes_trace])

In [ ]:
n = 400
x, z = swissroll(n)
W = knn_graph(distances(x), 10)
#W = eps_graph(distances_sq(x), 2.0)
plot_graph_3d(x, W, z[0])

In [ ]:
from ml_solutions import isomap

In [ ]:
# test code for profiling
# see https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html
#x = np.random.randn(2, 500)
#%prun y, W, DD = isomap(x, d=2, k=10)

In [ ]:
n = 100
x, z = swissroll(n)
k = 20
plot_graph_3d(x, knn_graph(distances(x), k), z[0]).show()
y, W, DD = isomap(x, d=2, k=k)
px.scatter(x=y[0], y=y[1], color=z[0])

<a class="anchor" id="lle"></a>
## 3. LLE - Local Linear Embedding

In [ ]:
n = 200
x, z = swissroll(200)

In [ ]:
# based on Sam Roweis implementation
# from https://cs.nyu.edu/~roweis/lle/code.html
def lle(x, d=2, k=9):
    # 1. find nearest neighbors
    d, n = x.shape[0], x.shape[1]
    dist = distances(x)       # non-squared distances
    dist.flat[::n+1] = np.infty  # you are not your neighbor!
    knn = dist.argpartition(k-1, axis=1)[:,:k]
    W = np.zeros((n,n))
    for i in range(n):
        W[i,knn[i]] = dist[i,knn[i]]
    plot_graph(x, W).show()
    # 2. solve for reconstruction weights
    if k>d:
        print('note: since k>d we regularize')
        tol = 1.0e-3
    else:
        tol = 0.0
    W = np.zeros((n, n))
    for i in range(n):
        z = x[:,knn[i]] - x[:,i].reshape(d, 1)
        C = z.T @ z
        C = C + np.eye(k)*tol*np.trace(C)
        w = np.linalg.solve(C, np.ones(k))
        W[i,knn[i]] = w / w.sum()
    # 3. compute embedding coordinates
    M = (np.eye(n) - W).T @ (np.eye(n) - W)
    Lambda, V = np.linalg.eigh(M)
    y = V[:,1:3].T * np.sqrt(n)
    return y, W
y, W = lle(x, d=2, k=9)
plot_graph_3d(x, W).show()   # check the graph
px.scatter(x=y[0], y=y[1], color=z[0])

<a class="anchor" id="mvu"></a>
## 4. MVU - maximum variance unfolding

STILL MISSING

In [ ]:
np.diag(np.diag(np.random.rand(5,5)))